In [ ]:
!pip install --upgrade pip
!pip install -Uqq fastapi pyngrok uvicorn python-multipart pdfminer.six transformers[sentencepiece] auto-gptq sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 k

# Authenticate ngrok

In [ ]:
!ngrok config add-authtoken 2WXWTsDTHS7fchamHbXVlcNyvur_56bYU7JCF67SaEh3NT1ZR

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Global arguments

In [ ]:
global FNAME
global TOP_K
global WINDOW_SIZE
global STEP_SIZE

global paragraphs
global embedding_model
global cross_encoder
global embeddings

# Model import

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig


In [ ]:
!GITHUB_ACTIONS=true

In [ ]:
model_name_or_path = "TheBloke/Luna-AI-Llama2-Uncensored-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=False,
        quantize_config=None)

# Text extraction

In [ ]:
from pdfminer.high_level import extract_text

def extract(fname, window_size, step_size):

    # extract text
    text = extract_text(fname)
    text = " ".join(text.split())

    # split into tokens
    text_tokens = text.split()

    sentences = []
    for i in range(0, len(text_tokens), step_size):
        window = text_tokens[i: i+window_size]
        if len(window) < window_size:
            break
        sentences.append(window)

    paragraphs = [" ".join(s) for s in sentences]
    print(paragraphs)
    return paragraphs

# Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util

In [ ]:
# Embedding the extracted text paragraphs

def embed(paragraphs):

    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
    model.max_seq_length = 512

    cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

    embeddings = model.encode(
        paragraphs,
        show_progress_bar=True,
        convert_to_tensor=True
    )

    return model, cross_encoder, embeddings

In [ ]:
# semantic search of text embeddings

def search(query, model, cross_encoder, embeddings, paragraphs, top_k):

    query_embeddings = model.encode(query, convert_to_tensor=True)
    query_embeddings = query_embeddings.cuda()
    hits = util.semantic_search(
        query_embeddings,
        embeddings,
        top_k=top_k,
    )[0]

    cross_input = [[query, paragraphs[hit["corpus_id"]]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_input)

    for idx in range(len(cross_scores)):
        hits[idx]["cross_score"] = cross_scores[idx]

    results = []
    hits = sorted(hits, key=lambda x: x["cross_score"], reverse=True)

    for hit in hits[:5]:
        results.append(paragraphs[hit["corpus_id"]].replace("\n", " "))

    return results

In [ ]:
prompt = "what are the differences between binary attributes and ordinal attributes "

def generate_reply(prompt):

    # Arguments
    FNAME = "uploads/text.pdf"
    TOP_K = 32
    WINDOW_SIZE = 128
    STEP_SIZE = 100

    # Extract text and Embeddings
    paragraphs = extract(FNAME, WINDOW_SIZE, STEP_SIZE)
    embedding_model, cross_encoder, embeddings = embed(paragraphs)

    context = search(
        prompt,
        embedding_model,
        cross_encoder,
        embeddings,
        paragraphs,
        TOP_K,
    )

    prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant uses the given context to answer all of the users queries. Restrict your answer only to the given context. Ensure that your answer is only answering the given USER prompt. Do not give extra information that is not asked. Try to use only the given context for answering the prompt. Always restrict your answer to a maximum of 300 words.

    CONTEXT: {context}
    USER: {prompt}. Answer only the given question. Do not answer anything else. Be as accurate as possible using the given context. If the question can be answered in a single sentence answer it in a single sentence.
    ASSISTANT:
    '''

    print("Reply: \n")

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

    ans = tokenizer.decode(output[0])
    ans = ans[ans.find('ASSISTANT:')+10:]
    ans = ans[:-4]

    print(ans)
    return ans

# Server

In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import shutil
import os

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Define a directory where uploaded PDF files will be saved
UPLOAD_DIR = "uploads"

if not os.path.exists(UPLOAD_DIR):
    os.makedirs(UPLOAD_DIR)



# API ENDPOINT FOR UPLOADING PDF  ----------------------------------------------

@app.post('/upload-pdf/')
async def upload_pdf(file: UploadFile):


    # Error handling
    if not file.filename.endswith(".pdf"):
        return {"error": "Only PDF files are allowed."}


    # Save the uploaded PDF file with the new name
    new_filename = "text.pdf"
    file_path = os.path.join(UPLOAD_DIR, new_filename)

    with open(file_path, "wb") as f:
        shutil.copyfileobj(file.file, f)

    return {
        "message": "File uploaded successfully",
        "extracted_text": "drama"
    }



# API ENDPOINT FOR PROCESSING TEXT  --------------------------------------------

@app.post('/process-text/')
async def process_text(input_data: dict):
    prompt = input_data.get('prompt')
    if not prompt:
        raise HTTPException(status_code=400, detail="prompt text not provided")

    prompt = str(prompt)

    reply = generate_reply(prompt)
    # reply = "This is reply to - " + prompt

    print("\n\nPROMPT:\n" + prompt)
    print("\n\nREPLY:\n" + reply)

    return {"reply": reply}



# ROOT API ENDPOINT ------------------------------------------------------------

@app.get('/')
async def root():
    return {'hello': 'world'}

import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


Exception in thread Thread-26 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 140, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Public URL: https://5aa1-35-225-128-54.ngrok-free.app


INFO:     Started server process [159]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     136.233.9.98:0 - "GET / HTTP/1.1" 200 OK
INFO:     136.233.9.98:0 - "POST /upload-pdf HTTP/1.1" 307 Temporary Redirect
INFO:     136.233.9.98:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK
INFO:     136.233.9.98:0 - "OPTIONS /process-text HTTP/1.1" 200 OK
INFO:     136.233.9.98:0 - "POST /process-text HTTP/1.1" 307 Temporary Redirect
INFO:     136.233.9.98:0 - "OPTIONS /process-text/ HTTP/1.1" 200 OK
['MGT1022 - LEAN START-UP MANAGEMENT Slot: TBB2 ASSIGNMENT - 3 CRITICALLY EXAMINE THE CSR ACTIVITIES PERFORMED BY ONE INDIAN AND ONE GLOBAL ORGANIZATION Submitted by: M. ANANYA RAJU [20BCE0537] PILLI SAI NISHANTH [20BCE0906] Submitted to: Prof. Jambeswar Sahu SMEC VIT, Vellore FALL SEMESTER 2023-24 CORPORATE SOCIAL RESPONSIBILITY ACTIVITIES responsibility (CSR) What is CSR? Corporate social is deﬁned as a self-regulatory business model that helps a ﬁrm/enterprise to be socially accountable - to itself, its stakeholders, and the public. By practicing CSR, also known as corporate citizens

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Reply: 



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



    
Starbucks CSR policies focus on community development, ethical sourcing, and environmental sustainability. They have implemented various initiatives such as community stores, partnerships with non-profit organizations, and sustainable practices to create a positive impact on society.


PROMPT:
Write a paragraph about Starbuck's CSR policies in exactly 60 words


REPLY:

    
Starbucks CSR policies focus on community development, ethical sourcing, and environmental sustainability. They have implemented various initiatives such as community stores, partnerships with non-profit organizations, and sustainable practices to create a positive impact on society.
INFO:     136.233.9.98:0 - "POST /process-text/ HTTP/1.1" 200 OK
INFO:     136.233.9.98:0 - "POST /upload-pdf HTTP/1.1" 307 Temporary Redirect
INFO:     136.233.9.98:0 - "POST /upload-pdf/ HTTP/1.1" 200 OK
INFO:     136.233.9.98:0 - "POST /process-text HTTP/1.1" 307 Temporary Redirect
['HAN 12-ch05-187-242-9780123814791 2011/6/1 

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Reply: 


    1. Ranking cubes for efficient top-k query processing in large relational data sets.
    2. Sampling cubes for multidimensional analysis on sample data.
    3. Prediction cubes for multidimensional data mining that facilitate predictive modeling.


PROMPT:
List the types of processes for advanced kinds of queries by exploring cube technology. 


REPLY:

    1. Ranking cubes for efficient top-k query processing in large relational data sets.
    2. Sampling cubes for multidimensional analysis on sample data.
    3. Prediction cubes for multidimensional data mining that facilitate predictive modeling.
INFO:     136.233.9.98:0 - "POST /process-text/ HTTP/1.1" 200 OK
